![Partners](../images/partners.png)

# Wind Power Output

This notebook calculates wind power generation and analyzes ramping events using wind speed forecasts.

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from ecmwf.opendata import Client
import geopandas as gpd

import warnings
warnings.filterwarnings("ignore")

## 1. Get Wind Power Plants Data


In [ ]:
# Select country to visualize
selected_country = "Kenya"  # Change this to any country 

gdf = gpd.read_file("/data/shared/renewable-energy-data/RePP/RePP_short_FeaturesToJSON.geojson")

# Filter data for selected country
country_data = gdf[gdf.country == selected_country]

# list of all operating wind power plants in the selected country   
wind_plants = country_data[(country_data['RE_source'] == 'Wind power') & (country_data['stat_ele'] == 'O')].sort_values(by='g_cap_mw', ascending=False)

In [ ]:
# example choose first plant
plant = wind_plants.iloc[0]
location_name = plant['HPP_name']
lat_point = plant['lat']
lon_point = plant['lon']
g_cap_mw = plant['g_cap_mw']

# print
print(f"Selected Plant: {location_name}, Capacity: {g_cap_mw} MW, Location: ({lat_point}, {lon_point})")

## 2. Get AIFS Forecast Data

Skip this step if the request is too large, forecast for 2025-11-06 run 00:00:00 is already downloaded in `data/AIFS/output/`. This is available for backup option in case of download issues.

In [ ]:
var_dict = {
    '100u': 'u100',   # 100m u-component of wind
    '100v': 'v100',   # 100m v-component of wind
}

area_bbox = [18, 32, -5, 52]  

# Parameters to download
parameters = ['100u', '100v']  

# Forecast steps (in hours, 6-hourly intervals up to 360 hours = 15 days)
steps = list(range(0, 60, 6))

# Output directory
oper_out_dir = Path("data/AIFS/output")
oper_out_dir.mkdir(parents=True, exist_ok=True)

client = Client(
    "ecmwf", 
    model="aifs-single",  # ECMWF's AI-based Integrated Forecast System
    resol="0p25",         # 0.25 degree resolution (~28 km)
)


for param in parameters:

    print(f"Downloading parameter: {param} ({var_dict.get(param, param)})")
    
    filename = 'tmp_oper.nc'
    
    # Retrieve operational forecast from latest run
    print("Retrieving data from ECMWF...")
    client.retrieve(
        date=0,           # Latest available date (0 = today)
        time=0,           # 00z run (can also use 6, 12, or 18 for other runs)
        step=steps,       # Forecast steps: 0, 6, 12, ..., 360 hours
        stream="oper",    # Operational high-resolution forecast
        type="fc",        # Forecast type
        levtype="sfc",    # Surface level
        param=param,      # Parameter to download
        target=filename   # Temporary file
    )
    print(f"Data downloaded to {filename}")

    print("Loading dataset...")
    ds = xr.open_dataset(filename, engine="cfgrib")

    # Convert step to timedelta and extract region
    ds['step'] = pd.to_timedelta(ds.step.values)
    ds = ds.sel(
        latitude=slice(area_bbox[0], area_bbox[2]), 
        longitude=slice(area_bbox[1], area_bbox[3])
    )

    # Get forecast initialization time
    forecast_datetime = ds.time.values
    forecast_datetime_str = pd.to_datetime(forecast_datetime).strftime("%Y%m%d%H%M%S")

    print(f"Forecast run time: {pd.to_datetime(forecast_datetime)}")
    print(f"Dataset shape: {ds[var_dict[param]].shape}")
    print(f"Time steps: {len(ds.step.values)}")

    # Save the extracted dataset
    out_fname = oper_out_dir / f'{forecast_datetime_str}-{steps[-1]}h_{param}.nc'
    ds.to_netcdf(out_fname)
    print(f"Saved to: {out_fname}")

    # Clean up temporary file
    Path(filename).unlink(missing_ok=True)

In [ ]:

# backup option
ds_100u = xr.open_dataset("/data/shared/renewable-energy-data/AIFS/output/20251106000000-360h_100u.nc")
ds_100v = xr.open_dataset("/data/shared/renewable-energy-data/AIFS/output/20251106000000-360h_100v.nc")

# compute magnitude
wind_speed = np.sqrt(ds_100u['u100']**2 + ds_100v['v100']**2)

# create dataset with variable "wind_speed" from dataarray
wind_speed_ds = wind_speed.to_dataset(name='wind_speed')

In [ ]:
# plot wind speed at location

from plot import plot_variable
plot_variable(
    wind_speed_ds.sel(step="18:00:00"),
    var_name='wind_speed',
    title='AIFS Wind Speed at 100m Height (m/s)',
    )


In [ ]:
forecast_location = wind_speed_ds['wind_speed'].sel(
    latitude=lat_point, 
    longitude=lon_point, 
    method='nearest'
)

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(forecast_location.valid_time, forecast_location, marker='o', label='Wind Speed Forecast', color='blue')
plt.title(f'Wind Speed Forecast at {location_name}')
plt.xlabel('Valid Time')
plt.ylabel('Wind Speed (m/s)')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# get daily mean values

# skip first value which is nan after diff
valid_times = forecast_location.valid_time[1:]

 # now get the daily sum from 6h intervals
forecast_mean_daily = forecast_location.groupby("valid_time.day").mean(dim="step")

# skip last day if incomplete
forecast_mean_daily = forecast_mean_daily[:-1]

valid_times_days = pd.date_range(
    start=pd.to_datetime(valid_times.values[0]).replace(hour=0, minute=0, second=0),
    end=pd.to_datetime(valid_times.values[-1]).replace(hour=0, minute=0, second=0),
    freq='D'
)[:-1]



In [ ]:
# plot daily mean of wind speed
plt.figure(figsize=(10, 4))
plt.plot(valid_times_days, forecast_mean_daily, marker='o', label='AIFS Daily mean Wind Speed', color='blue')
plt.title(f'Daily mean of Wind Speed at {location_name}')
plt.xlabel('Day')
plt.ylabel('Daily Wind Speed mean (m/s)')
plt.grid()
plt.legend()
plt.show()

## 3. Wind Power Model

The power curve follows a cubic relationship below rated speed and caps at installed capacity.

In [ ]:
def wind_power(
        v, 
        v_rated,
        C_wind=900.0
): 
    """
    Simple wind power model.
    Parameters:
    v : xarray.DataArray
        the forecasted wind speed at turbine hub height [m/s]
    v_rated : float
        the rated wind speed of the turbines [m/s]
    C_wind : float
        the total installed wind capacity [MW]
    """
    
    # Power curve (simplified)
    P_wind = xr.where(
        v < 0,
        0,
        xr.where(
            v < v_rated,
            C_wind * (v / v_rated) ** 3,
            C_wind
        )
    )

    return P_wind

In [ ]:
# Use daily sum of wind speed to calculate power forecast
power_forecast = wind_power(
    forecast_mean_daily, 
    v_rated=25.0, 
    C_wind=g_cap_mw
)

plt.figure(figsize=(10, 4))
plt.plot(valid_times_days, power_forecast, marker='o', label='Wind Power Forecast',
            color='purple')
plt.title(f'Wind Power Forecast at {location_name}')
plt.xlabel('Valid Time')
plt.ylabel('Power Output (MW)')
plt.grid()
plt.legend()
plt.show()